### doc_entity
ESTABELECE RELAÇÕES ENTRE DOCUMENTOS E ENTIDADES NA BASE DE DADOS NO MYSQL. TRABALHAMOS COM O ACERVO **ANTONIO AZEREDO DA SILVEIRA, MINISTÉRIO DAS RELAÇÕES EXTERIORES**.  

AS ENTIDADES A SEREM TRABALHADAS SÃO:
* PAÍSES
* PESSOAS

In [1]:
import os
import sys
import time
import datetime
import numpy as np
import pandas as pd
import re

import pymysql
import getpass
from sshtunnel import SSHTunnelForwarder

from IPython.display import clear_output

import csv
import xlrd
import xlwt

In [2]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
define pastas de trabalho
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''

path = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Azeredo Papers/Processamento/text-learning-tools/inputs/"


#### Abre arquivo XLS

In [3]:
workbook = xlrd.open_workbook(path+'PESSOAS-do-Accessus.xls', on_demand = True)

#worksheet = workbook.sheet_by_index(0)
worksheet = workbook.sheet_by_name('Rel_DescritorEleito')

WARNING *** file size (1367936) not 512 + multiple of sector size (512)


Captura nomes de entidades pessoas retiradas da base do Accessus/CPDOC

In [4]:
names_list = []

for line in range(worksheet.nrows):
    number_name = str(worksheet.cell(line, 0).value)
    check_number = re.search('^\d',number_name)
    if check_number is not None:
        if line == worksheet.nrows-1: break
        name = str(worksheet.cell(line, 1).value)
        name = str.lower(name)
        names_list.append(name)

In [5]:
names_list[:10]

['a. a. castro lima',
 'a. a. saldanha júnior',
 'a. azevedo',
 'a. faria',
 'a. i. de lima',
 'a. j. peixoto de castro',
 'a. j. pinto ferraz',
 'a. lafaiete müller',
 'a. pereira  alves',
 'a. rocha lima']

In [11]:
#teste
names_list_test = ['geisel', 'azeredo']
#text_test = 'azeredo assinou o tratado. geisel não concordou com a atitude de azeredo'
#text_test.count(names_list_test[0])

### Inserção na base de dados

##### ENTIDADE: PESSOA

Alimenta a tabela person_doc, da nossa base no mysql, fazendo interseção de dados da tabela persons com a tabela docs.

In [12]:
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
alimenta a a tabela person_doc, da nossa base no mysql.
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
count = 0
#percentil = int(len(cursor.rowcount/100)

pass_sshkey = getpass.getpass()
pass_mysql = getpass.getpass()
with SSHTunnelForwarder(('200.20.164.147', 22),
                        ssh_private_key = "C:/Users/marcelo.ribeiro/Dropbox/A-Marcelo/Educação-Trabalho/2016-CPDOC/Remoto/marcelo_priv_rsa",
                        #ssh_private_key = '/home/rsouza/.ssh/id_rsa',
                        ssh_private_key_password = pass_sshkey,
                        ssh_username="marcelobribeiro",
                        #ssh_username="rsouza",
                        remote_bind_address=('127.0.0.1', 3306)) as server:
    
    conn = pymysql.connect(host='localhost', 
                           port=server.local_bind_port, 
                           user='marcelobribeiro', 
                           #user='rsouza',
                           passwd=pass_mysql,
                           db='CPDOC_AS',
                           use_unicode=True, 
                           charset="utf8")
    cur = conn.cursor()
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    insere dados na base mysql
    captura documentos da tabela docs, transfere para a tabela topic-doc e insere dados de tópicos
    '''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''''
    
    cur.execute("DROP TABLE IF EXISTS person_doc")
    cur.execute('''CREATE TABLE IF NOT EXISTS person_doc 
               (person_id INT(11), doc_id VARCHAR(31), person_count BIGINT(21)
               DEFAULT NULL)
               ENGINE=MyISAM DEFAULT CHARSET='utf8';''')

    cur.execute("SELECT id,body FROM CPDOC_AS.docs")
    data = cur.fetchall()
    for row in data:
        count += 1 # for break event
        if row is None: break
        text_id = row[0]
        text =  row[1]
        #text = text.split()
        doc_id = row[0]
        for name in names_list_test:
            person_id = 1
            person_count = text.count(name)
            query = "INSERT INTO person_doc VALUES (%s, %s, %s)"
            cur.execute(query, (person_id, doc_id, person_count))
        print(name, 'text_id', text_id)
        #print(doc_id, topic_id, topic_score)   
        cur.execute("ALTER TABLE person_doc ORDER BY doc_id ASC, person_count DESC")
        
        if count == 10: break #amostra de apenas 10 linhas

········
········
azeredo text_id rb_1974.04.17_doc_I-66
azeredo text_id onu_1974.03.12_doc_II-49
azeredo text_id onu_1974.03.12_doc_V-2
azeredo text_id be_1974.04.16_doc_VI-18
azeredo text_id be_1977.06.01_doc_II-21
azeredo text_id ag_1974.03.13_doc_III-7
azeredo text_id ag_1973.11.20_doc_VI-75
azeredo text_id d_1974.04.23_doc_XXXII-8
azeredo text_id d_1974.03.26_doc_XXXII-32
azeredo text_id d_1974.03.26_doc_XXII-9


##### ENTIDADE: LUGAR (PAÍS)

Alimenta a a tabela country_doc, da nossa base no mysql, fazendo interseção de dados da tabela countries com a tabela docs.